In [1]:
%cd ..

import importlib
import json
import math
import os
import socket
import sys
import time


internal_path = os.path.abspath(os.path.join('.'))
sys.path.append(internal_path)
sys.path.append(internal_path + '/datasets')
sys.path.append(internal_path + '/utils')

import datetime
import uuid
from argparse import ArgumentParser

import setproctitle
import torch
import numpy as np
import pandas as pd 
import json
import wandb

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
from matplotlib.collections import LineCollection



/pub/hofmann-scratch/glanzillo/ded


In [28]:
os.environ['WANDB_API_KEY'] = 'c7314b16e17009f66f3043c0b7968e1142054123'
wandb.login()

api = wandb.Api()
runs = api.runs(path='continually/DataEfficientDistillation', filters={'tags': 'useful'})
print("Loaded", len(runs), "runs")

Loaded 81 runs


In [29]:
dir(runs)

['QUERY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_include_sweeps',
 '_load_page',
 '_sweeps',
 'client',
 'convert_objects',
 'cursor',
 'entity',
 'filters',
 'index',
 'last_response',
 'length',
 'more',
 'next',
 'objects',
 'order',
 'per_page',
 'project',
 'update_variables',
 'variables']

In [30]:
dir(runs.objects[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attrs',
 '_base_dir',
 '_entity',
 '_exec',
 '_files',
 '_full_history',
 '_include_sweeps',
 '_repr_html_',
 '_sampled_history',
 '_state',
 '_summary',
 'client',
 'create',
 'delete',
 'dir',
 'display',
 'entity',
 'file',
 'files',
 'history',
 'id',
 'json_config',
 'lastHistoryStep',
 'load',
 'log_artifact',
 'logged_artifacts',
 'name',
 'path',
 'project',
 'save',
 'scan_history',
 'snake_to_camel',
 'state',
 'storage_id',
 'summary',
 'sweep',
 'to_html',
 'update',
 'upload_file',
 'url',
 'use_artifact',
 'used_artifacts',
 'user',
 'username',
 'wait_until_finished']

In [31]:
config = json.loads(runs.objects[10].json_config)
config.keys()

dict_keys(['mse', 'sgd', 'seed', 'alpha', 'debug', 'h_dim', 'device', 'exp_id', 'max_lr', 'min_lr', 'compile', 'dataset', 'mlp_dim', 'n_heads', 'seq_len', 'exp_name', 'head_dim', 'log_path', 'n_layers', 'data_root', 'n_workers', 'use_flash', 'eval_every', 'vocab_size', 'decay_steps', 'logger_type', 'temperature', 'wandb_notes', 'project_path', 'grad_clip_norm', 'log_ckpt_every', 'max_eval_steps', 'eval_batch_size', 'log_grads_every', 'max_train_steps', 'test_batch_size', 'train_batch_size', 'log_metrics_every', 'relative_log_path', 'tokens_per_second', 'wandb_entity_name', 'log_terminal_every', 'wandb_project_name', 'log_alignments_every', 'log_activations_every', 'teacher_checkpoint_path', 'gradient_accumulation_steps'])

In [32]:
'weighed-cka' in runs.objects[10].tags

False

In [33]:
h = runs.objects[0].history()
h.head()

,_step,config,_runtime,_timestamp,parameter count,parameter count without embedding,eval_batches,eval_tokens,eval_bytes,_train/tokens_seen,...,_eval/normalised_ppl,_eval/top1_acc,_eval/normalised_ppl_over_tokens,_eval/normalised_loss,_eval/normalised_ppl_over_gpuseconds,x_axes/gpu_seconds,_eval/normalised_loss_over_gpuseconds,_eval/ppl_over_gpuseconds,_eval/top10_acc,_time/eval
0,0,"{'ncols': 1, 'nrows': 1, 'path': 'media/table/...",10.999835,1.710839e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,11.021492,1.710839e+09,110615040.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,11.021584,1.710839e+09,NaN,85449216.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,14.984527,1.710839e+09,NaN,NaN,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,14.984796,1.710839e+09,NaN,NaN,NaN,4096000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
h.columns

Index(['_step', 'config', '_runtime', '_timestamp', 'parameter count',
       'parameter count without embedding', 'eval_batches', 'eval_tokens',
       'eval_bytes', '_train/tokens_seen', '_train/learning_rate_0',
       '_time/forward', '_time/train_step', '_time/total_step', '_train/cka',
       '_time/backward', '_train/fa', '_time/load_batch',
       '_time/tokens_per_second', '_train/loss', '_time/iterations_per_second',
       '_eval/total_loss', '_eval/top10_acc_over_gpuseconds', '_eval/ppl',
       '_eval/fa', '_eval/ppl_over_tokens',
       '_eval/normalised_loss_over_tokens', '_eval/top1_acc_over_tokens',
       'x_axes/n_tokens', '_eval/top1_acc_over_gpuseconds',
       '_eval/top10_acc_over_tokens', '_eval/cka', '_eval/loss',
       '_eval/normalised_ppl', '_eval/top1_acc',
       '_eval/normalised_ppl_over_tokens', '_eval/normalised_loss',
       '_eval/normalised_ppl_over_gpuseconds', 'x_axes/gpu_seconds',
       '_eval/normalised_loss_over_gpuseconds', '_eval/ppl_over_g

In [35]:
list(h[6:]['_eval/cka'])

[nan,
 0.6270333528518677,
 nan,
 0.6837803721427917,
 nan,
 0.7097541093826294,
 nan,
 0.7144049406051636,
 nan,
 0.7308736443519592,
 nan,
 0.7452026009559631,
 nan,
 0.754500687122345,
 nan,
 0.768601655960083,
 nan,
 0.7724509239196777,
 nan,
 0.7881096601486206,
 nan,
 0.7960778474807739,
 nan,
 0.8032061457633972,
 nan,
 0.802556037902832,
 nan,
 0.8076269626617432,
 nan,
 0.8157808780670166,
 nan,
 0.8145785331726074,
 nan,
 0.8178026080131531,
 nan,
 0.827684760093689,
 nan,
 0.8257328271865845,
 nan,
 0.831561267375946,
 nan,
 0.8389953374862671,
 nan,
 0.8342667818069458,
 nan,
 0.8389791250228882,
 nan,
 0.8325879573822021,
 nan,
 0.8369709253311157,
 nan,
 0.832046389579773]

In [36]:
h['_step'].values[-1]

13000

In [37]:

COLUMNS = ['alpha','seed','student_net','teacher_net','step',
           'tokens_seen','cka_val','cka_train','fa_val','fa_train',
           'val_acc_top1', 'val_acc_top10', 'loss_train', 'ppl_val',
           'temperature', 'total_steps','w-cka']
res_df = pd.DataFrame(columns=COLUMNS)

skipped = 0
for run in runs.objects:
    h = run.history()[6:] # first rows are for validation

    try:
        config = json.loads(run.json_config)
        total = len(h)
        new_runs = '_eval/cka' in h.columns

        #figuring out teacher architecture
        teacher_path = config['teacher_checkpoint_path']['value'].split("/")[-1]
        if teacher_path == "mini":
            teacher = "mini"
        elif teacher_path == "gpt-mini":
            teacher = "gpt-mini"
        elif teacher_path == "gpt-medium":
            teacher = "medium"
        elif teacher_path in ["gpt-small","small"]:
            teacher = "small"
        else: raise Exception(f"{teacher_path} Unknown teacher.")

        #figuring out student architecture
        if config['h_dim']['value']==512:
            student="mini"
        elif config['h_dim']['value']==1024:
            if config['n_layers']['value']==4: 
                student="mini2"
            else: student="medium"
        elif config['h_dim']['value']==768: 
            student="small"

        try: T = config['temperature']['value']
        except KeyError: T=1 
        sub_df = pd.DataFrame({"alpha":[config['alpha']['value']]*total,
                            "seed":[config['seed']['value']]*total,
                            "total_steps":[h['_step'].values[-1]]*total,
                            "batch_size":[config['train_batch_size']['value']]*total,
                            "temperature":[T]*total,
                            "student_net":[student]*total,
                            "teacher_net":[teacher]*total,
                            "step":list(h['_step']),
                            "tokens_seen":list(h['_train/tokens_seen']),
                            "cka_val": list(h['_eval/cka']) if new_runs else [-1]*total ,
                            "cka_train":list(h['_train/cka']) if new_runs else [-1]*total,
                            "fa_val":list(h['_eval/fa']) if new_runs else [-1]*total,
                            "fa_train":list(h['_train/fa']) if new_runs else [-1]*total,
                            "val_acc_top1":list(h['_eval/top1_acc']),
                            "val_acc_top10":list(h['_eval/top1_acc']),
                            "loss_train":list(h['_train/loss']),
                            "ppl_val":list(h['_eval/ppl']),
                            "w-cka":['weighed-cka' in run.tags]*total
                            })
        
    except KeyError: 
        print("Skipping")
        skipped+=1
        continue # skipping the runs without values
    except IndexError: 
        print("Skipping")
        skipped+=1
        continue # skipping the runs without values

    res_df = pd.concat([res_df,sub_df], ignore_index=True)

print(f"Completion. {skipped} runs skipped.")
res_df

/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast 

Skipping
Skipping


/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast 

Skipping


/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast 

...

Skipping


/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)
/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast 

Completion. 5 runs skipped.


/tmp/ipykernel_296345/2622115055.py:69: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  res_df = pd.concat([res_df,sub_df], ignore_index=True)


,alpha,seed,student_net,teacher_net,step,tokens_seen,cka_val,cka_train,fa_val,fa_train,val_acc_top1,val_acc_top10,loss_train,ppl_val,temperature,total_steps,w-cka,batch_size
0,1,21,small,small,400,52559872.0,NaN,0.584778,NaN,-0.00189,NaN,NaN,4.392101,NaN,1,13000,False,256.0
1,1,21,small,small,500,NaN,0.627033,NaN,-0.008474,NaN,0.242307,0.242307,NaN,83.830728,1,13000,False,256.0
2,1,21,small,small,800,104988672.0,NaN,0.636134,NaN,-0.00362,NaN,NaN,3.892153,NaN,1,13000,False,256.0
3,1,21,small,small,1000,NaN,0.683780,NaN,-0.006513,NaN,0.299115,0.299115,NaN,52.191455,1,13000,False,256.0
4,1,21,small,small,1200,157417472.0,NaN,0.638793,NaN,-0.00099,NaN,NaN,3.608682,NaN,1,13000,False,256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7182,1,55,mini,medium,9600,629211136.0,0.771892,0.715811,-1.000000,-1.00000,0.324176,0.324176,3.514033,42.103190,1,10000,False,128.0
7183,1,55,mini,medium,9700,635764736.0,0.768522,0.704897,-1.000000,-1.00000,0.324454,0.324454,3.463334,42.055484,1,10000,False,128.0
7184,1,55,mini,medium,9800,642318336.0,0.777276,0.722243,-1.000000,-1.00000,0.325192,0.325192,3.539261,41.852502,1,10000,False,128.0
7185,1,55,mini,medium,9900,648871936.0,0.778837,0.723983,-1.000000,-1.00000,0.325411,0.325411,3.509279,41.864547,1,10000,False,128.0


In [38]:
logs_path = './logs/results/languini_results'
res_df.to_pickle(logs_path)

In [39]:
pd.read_pickle(logs_path)

,alpha,seed,student_net,teacher_net,step,tokens_seen,cka_val,cka_train,fa_val,fa_train,val_acc_top1,val_acc_top10,loss_train,ppl_val,temperature,total_steps,w-cka,batch_size
0,1,21,small,small,400,52559872.0,NaN,0.584778,NaN,-0.00189,NaN,NaN,4.392101,NaN,1,13000,False,256.0
1,1,21,small,small,500,NaN,0.627033,NaN,-0.008474,NaN,0.242307,0.242307,NaN,83.830728,1,13000,False,256.0
2,1,21,small,small,800,104988672.0,NaN,0.636134,NaN,-0.00362,NaN,NaN,3.892153,NaN,1,13000,False,256.0
3,1,21,small,small,1000,NaN,0.683780,NaN,-0.006513,NaN,0.299115,0.299115,NaN,52.191455,1,13000,False,256.0
4,1,21,small,small,1200,157417472.0,NaN,0.638793,NaN,-0.00099,NaN,NaN,3.608682,NaN,1,13000,False,256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7182,1,55,mini,medium,9600,629211136.0,0.771892,0.715811,-1.000000,-1.00000,0.324176,0.324176,3.514033,42.103190,1,10000,False,128.0
7183,1,55,mini,medium,9700,635764736.0,0.768522,0.704897,-1.000000,-1.00000,0.324454,0.324454,3.463334,42.055484,1,10000,False,128.0
7184,1,55,mini,medium,9800,642318336.0,0.777276,0.722243,-1.000000,-1.00000,0.325192,0.325192,3.539261,41.852502,1,10000,False,128.0
7185,1,55,mini,medium,9900,648871936.0,0.778837,0.723983,-1.000000,-1.00000,0.325411,0.325411,3.509279,41.864547,1,10000,False,128.0
